In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood <br/>
Only process the cells that have an assigned borough. <br/>Ignore cells with a borough that is Not assigned.<br/>
More than one neighborhood can exist in one postal code area. <br/>For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. <br/>These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.<br/>
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.<br/>
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.<br/>
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [3]:
Postal_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wikipedia = requests.get(Postal_url).text

In [4]:
soup = BeautifulSoup(wikipedia, 'xml')
table=soup.find('table')
column_names = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [5]:
#generates the table and merges the neighborhoods

for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [6]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
df.clean=df[df['Borough']!='Not assigned']
df.clean.head()

#removes the redundant places

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if __name__ == '__main__':


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.clean.shape

(103, 3)

In [10]:
def get_geocode(postal_code):
    # initialize
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [11]:
geospatial=pd.read_csv('http://cocl.us/Geospatial_data')

In [12]:
geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
geospatial_complete = pd.merge(geospatial, df.clean, on='Postalcode')

KeyError: 'Postalcode'